In [1]:
import pandas as pd
from typing import List
import re
import unicodedata

# Entendimento do Negócio

### Qual é o problema de negócio que queremos resolver com esses dados?
Queremos melhorar o processo de associação de candidatos com vagas
### Qual o objetivo final do modelo que desenvolveremos?
Queremos descobrir qual o candidato com o melhor match para cada vaga
### Quais são as principais entidades ou conceitos que esses dados representam?

### Há alguma métrica de sucesso específica que precisamos otimizar ou prever?

### Existem regras de negócio ou restrições que eu preciso conhecer?

### Quem são os usuários do modelo e como ele será utilizado?

# Exploração dos Dados

In [2]:
# carrega tabelas em dataframes
df_applicants = pd.read_json('data/raw/applicants.json', orient='index')
df_prospects = pd.read_json('data/raw/prospects.json', orient='index')
df_vagas = pd.read_json('data/raw/vagas.json', orient='index')

## Tabelas

Apllicants -> informações dos candidatos, tanto pessoais como profissionais

Prospects -> informações dos candidatos inscritos ou relacionados a cada uma das vagas

Vagas -> informações das vagas

### Tabela Applicants

In [3]:
df_applicants.head()

,infos_basicas,informacoes_pessoais,informacoes_profissionais,formacao_e_idiomas,cargo_atual,cv_pt,cv_en
31000,"{'telefone_recado': '', 'telefone': '(11) 9704...",{'data_aceite': 'Cadastro anterior ao registro...,"{'titulo_profissional': '', 'area_atuacao': ''...","{'nivel_academico': '', 'nivel_ingles': '', 'n...",{},assistente administrativo\n\n\nsantosbatista\n...,
31001,"{'telefone_recado': '', 'telefone': '(11) 9372...",{'data_aceite': 'Cadastro anterior ao registro...,{'titulo_profissional': 'Analista Administrati...,{'nivel_academico': 'Ensino Superior Incomplet...,{},formação acadêmica\nensino médio (2º grau) em ...,
31002,"{'telefone_recado': '', 'telefone': '(11) 9239...",{'data_aceite': 'Cadastro anterior ao registro...,{'titulo_profissional': 'Administrativo | Fina...,{'nivel_academico': 'Ensino Superior Completo'...,{},objetivo: área administrativa | financeira\n\n...,
31003,"{'telefone_recado': '', 'telefone': '(11) 9810...",{'data_aceite': 'Cadastro anterior ao registro...,"{'titulo_profissional': 'Área administrativa',...",{'nivel_academico': 'Ensino Superior Incomplet...,{},formação\nensino médio completo\ninformática i...,
31004,"{'telefone_recado': '', 'telefone': '(11) 9251...",{'data_aceite': 'Cadastro anterior ao registro...,"{'titulo_profissional': '', 'area_atuacao': ''...","{'nivel_academico': '', 'nivel_ingles': '', 'n...",{},última atualização em 09/11/2021\n­ sp\n\nensi...,


In [4]:
# normalização dos dicionários contidos nas colunas
df_infos_basicas = pd.json_normalize(df_applicants['infos_basicas'])
df_informacoes_pessoais = pd.json_normalize(df_applicants['informacoes_pessoais'])
df_informacoes_profissionais = pd.json_normalize(df_applicants['informacoes_profissionais'])
df_formacao_e_idiomas = pd.json_normalize(df_applicants['formacao_e_idiomas'])
df_cargo_atual = pd.json_normalize(df_applicants['cargo_atual'])

# recriando o df original com todos os dados
#df_applicants.drop(['cv_pt', 'cv_en'], axis=1, inplace=True)
df_applicants = pd.concat([df_applicants.drop('infos_basicas', axis=1), df_infos_basicas], axis=1)
df_applicants = pd.concat([df_applicants.drop('informacoes_pessoais', axis=1), df_informacoes_pessoais], axis=1)
df_applicants = pd.concat([df_applicants.drop('informacoes_profissionais', axis=1), df_informacoes_profissionais], axis=1)
df_applicants = pd.concat([df_applicants.drop('formacao_e_idiomas', axis=1), df_formacao_e_idiomas], axis=1)
df_applicants = pd.concat([df_applicants.drop('cargo_atual', axis=1), df_cargo_atual], axis=1)

In [5]:
pd.set_option('display.max_columns', None)
df_applicants.head()

,cv_pt,cv_en,telefone_recado,telefone,objetivo_profissional,data_criacao,inserido_por,email,local,sabendo_de_nos_por,data_atualizacao,codigo_profissional,nome,data_aceite,nome,cpf,fonte_indicacao,email,email_secundario,data_nascimento,telefone_celular,telefone_recado,sexo,estado_civil,pcd,endereco,skype,url_linkedin,facebook,download_cv,titulo_profissional,area_atuacao,conhecimentos_tecnicos,certificacoes,outras_certificacoes,remuneracao,nivel_profissional,qualificacoes,experiencias,nivel_academico,nivel_ingles,nivel_espanhol,outro_idioma,instituicao_ensino_superior,cursos,ano_conclusao,outro_curso,id_ibrati,email_corporativo,cargo_atual,projeto_atual,cliente,unidade,data_admissao,data_ultima_promocao,nome_superior_imediato,email_superior_imediato
31000,assistente administrativo\n\n\nsantosbatista\n...,,,(19) 94846-9471,CWF - SANTANDER INDICAÇÃO SUBST,02-08-2018 09:29:57,pelo próprio candidato,sr._anthony_gomes@hotmail.com,,Outros,02-08-2018 09:29:57,12097,Sr. Anthony Gomes,Cadastro anterior ao registro de aceite,Sr. Anthony Gomes,,Outros: Importação,sr._anthony_gomes@hotmail.com,,0000-00-00,(19) 94846-9471,,,,,,,,,NaN,CWF - SANTANDER INDICAÇÃO SUBST,,,,,0,,NaN,NaN,,,,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31001,formação acadêmica\nensino médio (2º grau) em ...,,,(24) 96341-6187,CONSULTORA ABAP SR,02-08-2018 10:45:46,pelo próprio candidato,dra._ana_sophia_guerra@gmail.com,,Outros,02-08-2018 10:45:46,12098,Dra. Ana Sophia Guerra,Cadastro anterior ao registro de aceite,Dra. Ana Sophia Guerra,,Outros: Importação,dra._ana_sophia_guerra@gmail.com,,0000-00-00,(24) 96341-6187,,,,,,,,,NaN,CONSULTORA ABAP SR,,,,,0,,,,,,,-,NaN,NaN,NaN,Outro Curso:,,,,,,,,,,
31002,objetivo: área administrativa | financeira\n\n...,,,(13) 98463-7528,~Consultor SAP MMWM,02-08-2018 11:20:33,pelo próprio candidato,srta._maria_cecília_garcia@gmail.com,"Santos, São Paulo",Outros,02-08-2018 11:20:33,12099,Srta. Maria Cecília Garcia,Cadastro anterior ao registro de aceite,Srta. Maria Cecília Garcia,,Outros: Importação,srta._maria_cecília_garcia@gmail.com,,0000-00-00,(13) 98463-7528,,,,,são paulo,,,,NaN,~Consultor SAP MMWM,,• Consultor Funcional especializado em Materia...,,,0,,NaN,NaN,Pós Graduação Completo,,,-,Universidade Mackenzie,Administração de Empresas,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31003,formação\nensino médio completo\ninformática i...,,,011-9-8415-8195 97527-7215,Websphere/Middleware,02-08-2018 11:22:27,pelo próprio candidato,sra._melina_rios@hotmail.com,,Outros,02-08-2018 11:22:27,12100,Sra. Melina Rios,Cadastro anterior ao registro de aceite,Sra. Melina Rios,,Outros: Importação,sra._melina_rios@hotmail.com,,0000-00-00,011-9-8415-8195 97527-7215,,,,,,,,,NaN,Websphere/Middleware,,,,,0,,NaN,NaN,,,,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31004,última atualização em 09/11/2021\n­ sp\n\nensi...,,,(11) 98856-4298,CONSULTOR ABAP SR,02-08-2018 11:41:34,pelo próprio candidato,yasmin_da_mota@gmail.com,,Outros,02-08-2018 11:41:34,12101,Yasmin da Mota,Cadastro anterior ao registro de aceite,Yasmin da Mota,,Outros: Importação,yasmin_da_mota@gmail.com,,0000-00-00,(11) 98856-4298,,,,,,,,,NaN,CONSULTOR ABAP SR,,,,,0,,,,,,,-,NaN,NaN,NaN,Outro Curso:,,,,,,,,,,


### Colunas da tabela Applicants

- id: chave primária da tabela
- telefone_recado: telefone de recado
- telefone: telefone pessoal
- objetivo_profissional: vaga que o candidato pretende
- data_criacao: data de criação do registro na plataforma
- inserido por: origem da informação na plataforma
- email: email do usuário
- local: local de moradia do usuário
- sabendo_de_nos_por: onde o usuário tomou conhecimento
- data_atualizacao: data da última atualização na plataforma
- codigo_profissional: código do profissional dentro da plataforma -> está relacionado com outra tabela?
- nome: nome completo
- data_aceite: -> verificar
- nome: nome completo
- cpf: cpf
- fonte_indicacao: fonte do cadastro do usuário. Pode ser por importação, site ou anúncio, ou até indicação de cliente/funcionário
- email: email
- email_secundario: -> email de contato secundario
- data_nascimento: data de nascimento do usuário
- telefone_celular: celular de contado do usuário
- telefone_recado: número de telefone para contato
- sexo: sexo do usuário
- estado_civil: estado civil do usuário
- pcd: flag para definir se o usuário é pcd
- endereco: endereço do usuário
- skype: contato de skype do usuário
- url_linkedin: url do linkedIn do usuário
- facebook: facebook do usuário
- download_cv: nome do arquivo do cv do usuário
- titulo_profissional: cargo atual do usuário
- area_atuacao: área da atuação atual do usuário
- conhecimentos_tecnicos: conhecimentos técnicos do usuário
- certificacoes: certificações que o usuário possui
- outras_certificacoes: outras certificações do usuário
- remuneracao: remuneração atual do usuário
- nivel_profissional: nível de senioridade do usuário
- qualificacoes: qualificações do usuário
- experiencias: experiências profissionais do usuário
- nivel_academico: nível acadêmico atual do usuário
- nivel_ingles: nível de inglês
- nivel_espanhol: nível de espanhol
- outro_idioma: outr idioma + nível correspondente
- instituicao_ensino_superior: instituição que o usuáro cursou o curso superior
- cursos: formação do usuário
- ano_conclusao: ano de conclusão do curso 
- outro_curso: vazio
- id_ibrati:
- email_corporativo: vazio. deve ser e-mail corporativo atual
- cargo_atual: cargo atual do usuário
- projeto_atual: projeto atual que o usuário está envolvido
- cliente: cliente em que o usuário está trabalhando
- unidade: unidade em que o usuário está trabalhando
- data_admissao: data de admissão do usuário no emprego atual
- data_ultima_promocao: data da última promoção do usuário
- nome_superior_imediato: nome do superior imediato
- email_superior_imediato: contato do superior imediato no projeto


In [6]:
# selecao de features relevantes para o problema a ser resolvido
features_applicants = ['pcd', 'area_atuacao', 'conhecimentos_tecnicos', 
                       'certificacoes', 'outras_certificacoes', 'qualificacoes', 
                       'experiencias', 'nivel_academico', 'nivel_ingles', 
                       'nivel_espanhol', 'outro_idioma', 'cursos', 'cargo_atual',
                       'data_admissao', 'data_ultima_promocao', 'cv_pt']
df_applicants = df_applicants[features_applicants]
df_applicants = df_applicants.reset_index()
df_applicants = df_applicants.rename(columns = {'index': 'id'})

### Tabela Prospects

Faz a associação da vaga com os possíveis candidatos

In [7]:
df_prospects.head(25)

,titulo,modalidade,prospects
4530,CONSULTOR CONTROL M,,"[{'nome': 'José Vieira', 'codigo': '25632', 's..."
4531,2021-2607395-PeopleSoft Application Engine-Dom...,,"[{'nome': 'Sra. Yasmin Fernandes', 'codigo': '..."
4532,,,[]
4533,2021-2605708-Microfocus Application Life Cycle...,,"[{'nome': 'Arthur Almeida', 'codigo': '26338',..."
4534,2021-2605711-Microfocus QTP - UFT Automation T...,,"[{'nome': 'Ana Luiza Vieira', 'codigo': '26361..."
4535,Microfocus QTP - UFT Automation Testing-Lead-2...,CLT,"[{'nome': 'Arthur Almeida', 'codigo': '26338',..."
4536,Gestão De Incidentes Field Support - 11755457,,"[{'nome': 'Sr. Cauã Vargas', 'codigo': '25362'..."
4537,PMO Practitioner - 11744940,,"[{'nome': 'Bárbara Nascimento', 'codigo': '252..."
4538,,,[]
4539,,,[]


In [8]:
# normalização dos dicionários contidos nas colunas
df_prospects_exploded = df_prospects.explode('prospects')
df_prospects_normalized = pd.json_normalize(df_prospects_exploded['prospects'])

# recriando o df original com todos os dados
df_prospects_exploded = df_prospects_exploded.drop('prospects', axis = 1).reset_index()
df_prospects = pd.concat([df_prospects_exploded, df_prospects_normalized], axis = 1)

# ajustando nomes das features
df_prospects.rename(columns = {'index': 'id_vaga', 'codigo': 'id_cand'}, inplace = True)

### Tabela Vagas

In [9]:
df_vagas.head()

,informacoes_basicas,perfil_vaga,beneficios
5185,"{'data_requicisao': '04-05-2021', 'limite_espe...","{'pais': 'Brasil', 'estado': 'São Paulo', 'cid...","{'valor_venda': '-', 'valor_compra_1': 'R$', '..."
5184,"{'data_requicisao': '04-05-2021', 'limite_espe...","{'pais': 'Brasil', 'estado': 'São Paulo', 'cid...","{'valor_venda': '-', 'valor_compra_1': 'R$', '..."
5183,"{'data_requicisao': '04-05-2021', 'limite_espe...","{'pais': 'Brasil', 'estado': 'São Paulo', 'cid...","{'valor_venda': '-', 'valor_compra_1': 'R$', '..."
5182,"{'data_requicisao': '04-05-2021', 'limite_espe...","{'pais': 'Brasil', 'estado': 'São Paulo', 'cid...","{'valor_venda': '- p/ mês (168h)', 'valor_comp..."
5181,"{'data_requicisao': '04-05-2021', 'limite_espe...","{'pais': 'Brasil', 'estado': 'São Paulo', 'cid...","{'valor_venda': '-', 'valor_compra_1': 'R$', '..."


In [10]:
# normalização dos dicionários contidos nas colunas
df_informacoes_basicas = pd.json_normalize(df_vagas['informacoes_basicas'])
df_perfil_vaga = pd.json_normalize(df_vagas['perfil_vaga'])
df_beneficios = pd.json_normalize(df_vagas['beneficios'])

# recriando o df original com todos os dados
df_vagas = pd.concat([df_vagas.drop('informacoes_basicas', axis=1), df_informacoes_basicas], axis=1)
df_vagas = pd.concat([df_vagas.drop('perfil_vaga', axis=1), df_perfil_vaga], axis=1)
df_vagas = pd.concat([df_vagas.drop('beneficios', axis=1), df_beneficios], axis=1)

In [11]:
pd.set_option('display.max_columns', None)
df_vagas.head()

,data_requicisao,limite_esperado_para_contratacao,titulo_vaga,vaga_sap,cliente,solicitante_cliente,empresa_divisao,requisitante,analista_responsavel,tipo_contratacao,prazo_contratacao,objetivo_vaga,prioridade_vaga,origem_vaga,superior_imediato,nome,telefone,data_inicial,data_final,nome_substituto,pais,estado,cidade,bairro,regiao,local_trabalho,vaga_especifica_para_pcd,faixa_etaria,horario_trabalho,nivel profissional,nivel_academico,nivel_ingles,nivel_espanhol,outro_idioma,areas_atuacao,principais_atividades,competencia_tecnicas_e_comportamentais,demais_observacoes,viagens_requeridas,equipamentos_necessarios,habilidades_comportamentais_necessarias,valor_venda,valor_compra_1,valor_compra_2
5185,23-11-2021,00-00-0000,Oracle - 12447747,Não,Nelson-Page,Dr. Raul Monteiro,Decision São Paulo,Cecília Freitas,Luna Correia,"CLT Full, PJ/Autônomo",Determinado,Contratação,Alta: Alta complexidade 3 a 5 dias,Nova Posição,Superior Imediato:,NaN,NaN,07-12-2021,06-12-2022,NaN,Brasil,Pernambuco,Recife,,,2000,Não,De: Até:,,Analista,Ensino Superior Completo,Avançado,Básico,,TI - Projetos-,"Desenhar, construir e configurar aplicações pa...",Habilidade principal: Aplicativos Oracle PLSQL...,Budgeted Rate - indicate currency and type (ho...,Não,,NaN,- p/ mês (168h),Fechado,
5184,23-11-2021,00-00-0000,E2E Consultant Copy 01 - 12447976,Não,Nelson-Page,Dr. Raul Monteiro,Decision São Paulo,Cecília Freitas,Emanuella Campos,"CLT Full, PJ/Autônomo",Determinado,Contratação,Alta: Alta complexidade 3 a 5 dias,Nova Posição,Superior Imediato:,NaN,NaN,08-12-2021,07-04-2022,NaN,Brasil,São Paulo,São Paulo,Chácara Santo Antônio,Sul,2000,Não,De: Até:,,Analista,Ensino Superior Completo,Básico,Básico,,TI - Projetos-,Projeto de Arquitetura - Revise e integre todo...,Outros detalhes do trabalho: 1 - SAS Campaign ...,Budgeted Rate - indicate currency and type (ho...,Não,,NaN,- p/ mês (168h),Fechado,
5183,23-11-2021,00-00-0000,Oracle - 12448269,Não,Nelson-Page,Dr. Raul Monteiro,Decision São Paulo,Cecília Freitas,Laura Pacheco,"CLT Full, PJ/Autônomo",Determinado,Contratação,Alta: Alta complexidade 3 a 5 dias,Nova Posição,Superior Imediato:,NaN,NaN,29-11-2021,28-05-2022,NaN,Brasil,Pernambuco,Recife,,,2000,Não,De: Até:,,Analista,Ensino Superior Completo,Intermediário,Básico,,TI - Projetos-,"Oracle Financials Cloud - Projete, crie e conf...",Outros detalhes do trabalho: 1 - Oracle Financ...,Budgeted Rate - indicate currency and type (ho...,Não,,NaN,- p/ mês (168h),Fechado,
5182,23-11-2021,00-00-0000,Oracle - 12448277,Não,Nelson-Page,Dr. Raul Monteiro,Decision São Paulo,Cecília Freitas,Clara Rios,"CLT Full, PJ/Autônomo",Determinado,Contratação,Alta: Alta complexidade 3 a 5 dias,Nova Posição,Superior Imediato:,NaN,NaN,07-12-2021,06-12-2022,NaN,Brasil,Pernambuco,Recife,,,2000,Não,De: Até:,,Analista,Ensino Superior Completo,Básico,Básico,,TI - Projetos-,"Desenhar, construir e configurar aplicações pa...",Habilidades Primárias: Aplicativos Oracle PLSQ...,Budgeted Rate - indicate currency and type (ho...,Não,,NaN,- p/ mês (168h),Fechado,
5181,23-11-2021,00-00-0000,4621328 - Scrum Master,Não,Miller-Curry,Guilherme Campos,Decision Campinas,Sara Porto,Sra. Joana Sousa,CLT Full,Indeterminado,,,,Superior Imediato:,,,NaN,NaN,NaN,Brasil,São Paulo,São Paulo,,,2000,Não,De: Até:,,Sênior,Ensino Superior Completo,Nenhum,Nenhum,,Gestão e Alocação de Recursos de TI-,Scrum Master,Scrum Master,,,,NaN,-,F2NT,


In [12]:
features_vagas = ['titulo_vaga', 'vaga_sap', 'cliente', 'solicitante_cliente',
                  'tipo_contratacao', 'local_trabalho', 'vaga_especifica_para_pcd',
                  'nivel profissional', 'nivel_academico', 'nivel_ingles', 'nivel_espanhol',
                  'outro_idioma', 'areas_atuacao', 'principais_atividades',
                  'competencia_tecnicas_e_comportamentais']
df_vagas = df_vagas[features_vagas]
df_vagas = df_vagas.reset_index()
df_vagas = df_vagas.rename(columns = {'index': 'id'})

### Colunas da tabela Vagas
- data_requicisao: data de requisição da vaga
- limite_esperado_para_contratacao: data limite para contratação
- titulo_vaga: título da vaga e código correspondente
- vaga_sap: vaga em um projeto sap
- cliente: nome do cliente que solicitou a vaga
- solicitante_cliente: nome do solicitante da empresa cliente
- empresa_divisao: divisão da empresa que cuida da vaga
- requisitante: nome do requisitante da vaga
- analista_responsavel: nome do analista responsável pela vaga
- tipo_contratacao: tipo de contratação. Pode ser um ou mais
- prazo_contratacao: define se há ou não um prazo para a contratação
- objetivo_vaga: objetivo da vaga, como contratação, prospecção, parcerias
- prioridade_vaga: nível de urgência da vaga
- origem_vaga: nova posição ou substituição
- superior_imediato: nome do superior imediato. Não há nenhum preenchido
- nome: 
- telefone: 
- data_inicial: data inicial do processo da vaga
- data_final: data final do processo da vaga
- nome_substituto: 
- pais: país da vaga
- estado: estado da vaga
- cidade: cidade da vaga
- bairro: bairro da vaga
- regiao: região da vaga
- local_trabalho: local de trabalho. 1000, 2000? O que significa?
- vaga_especifica_para_pcd: especifica se a vaga é somente para pcd 
- faixa_etaria: faixa etária da vaga
- horario_trabalho: horário de trabalho da vaga
- nivel profissional: nível de senioridade necessário para a vaga
- nivel_academico: nível acadêmico necessário para a vaga
- nivel_ingles: nível de inglês necessário para a vaga
- nivel_espanhol: nível de espanhol necessário para a vaga
- outro_idioma: nível necessário de outro idioma
- areas_atuacao: área de atuação da vaga
- principais_atividades: principais atividades a serem exercidas na vaga
- competencia_tecnicas_e_comportamentais: competências necessárias para a vaga
- demais_observacoes: observações adicionais
- viagens_requeridas: define se as viagens são requeridas ou não
- equipamentos_necessarios: equipamentos necessários para a vaga
- habilidades_comportamentais_necessarias: habilidades comportaentais necessárias. Na verdade parece um campo bem estranho
- valor_venda: valor de venda do projeto de consultoria
- valor_compra_1: valor de salário e ou benefício
- valor_compra_2: informações adicionais de pagamento

## Pré-Processamento dos Dados e Feature Engineering

In [13]:
from sklearn.preprocessing import OrdinalEncoder


def nivel_idioma(df: pd.DataFrame, language_features: List[str]) -> None:
    '''Trata dados faltantes de idiomas e utiliza o Ordinal Encoder para categorizar os níveis de idiomas'''
    language_order = ['Nenhum', 'Básico', 'Intermediário', 'Avançado', 'Fluente']
    enc = OrdinalEncoder(categories=[language_order])

    for language in language_features:
        df[f'{language}'] = df[f'{language}'].fillna('Nenhum')
        df[f'{language}'] = df[f'{language}'].replace('', 'Nenhum')
        # para igualar com o nível dos candidatos
        df[f'{language}'] = df[f'{language}'].replace('Técnico', 'Avançado')

        df[f'{language}_encoded'] = enc.fit_transform(df[[f'{language}']])
        
    df[f'{language}'].drop(columns = language_features, inplace = True)

    return None


def nivel_educacao(df: pd.DataFrame) -> None:
    '''Trata dados faltantes de nível de educaão e utiliza o Ordinal Encoder para categorizar os níveis de educação'''
    education_order = ['', 'ensino fundamental incompleto', 'ensino fundamental completo', 
                    'ensino medio incompleto', 'ensino medio completo',
                    'ensino tecnico incompleto', 'ensino tecnico completo',
                    'ensino superior incompleto', 'ensino superior completo',
                    'pos graduacao incompleto', 'pos graduacao completo',
                    'mestrado incompleto', 'mestrado completo',
                    'doutorado incompleto', 'doutorado completo']
    
    df['nivel_academico'] = df['nivel_academico'].astype(str).str.lower().str.strip().replace('nan', '')
    df['nivel_academico'] = df['nivel_academico'].apply(lambda x: re.sub(r'\bcursando\b', 'incompleto', x) if x else x)

    enc_ed = OrdinalEncoder(categories=[education_order])
    df['nivel_academico_encoded'] = enc_ed.fit_transform(df[['nivel_academico']])
    df.drop(columns = ['nivel_academico'], inplace = True)

    return 


def padroniza_texto (df: pd.DataFrame, features_list: List[str]) -> None:
    '''Remove espaços, torna todas as letras minúsculas e remove caracteres especiais dos campos texto'''
    for feature in features_list:
        df[f'{feature}'] = df[f'{feature}'].str.lower().str.strip()
        # Normalize accented characters (e.g., 'á' becomes 'a', 'ç' becomes 'c')
        # This converts accented characters into their base form plus a diacritic,
        # then removes the diacritic.
        df[feature] = df[feature].apply(lambda x: unicodedata.normalize('NFKD', str(x)).encode('ascii', 'ignore').decode('utf-8'))
        # Remove special characters (keep only letters, numbers, and spaces)
        df[feature] = df[feature].apply(lambda x: re.sub(r'[^a-zA-Z0-9\s]', '', str(x)))

    return None

In [14]:
# calculo de feature de experiência
df_applicants['tempo_exp'] = (pd.to_datetime('2025-06-01') - 
                                             pd.to_datetime(df_applicants['data_admissao'], 
                                                            dayfirst=True, 
                                                            errors='coerce')).dt.days/365.25
# considerando que valores faltantes são candidatos sem experiência
df_applicants['tempo_exp'] = df_applicants['tempo_exp'].fillna(0)
# mas e se o usuário estiver desempregado? essa suposição é razoavel?

In [15]:
# tratamento das colunas de texto
features_applicants = ['area_atuacao', 'conhecimentos_tecnicos', 
                       'certificacoes', 'outras_certificacoes', 'qualificacoes', 
                       'experiencias', 'nivel_academico', 'outro_idioma', 
                       'cursos', 'cargo_atual', 'cv_pt']
padroniza_texto(df_applicants, features_applicants)
features_vagas = ['titulo_vaga', 'vaga_sap', 'cliente', 'solicitante_cliente',
                  'local_trabalho', 'vaga_especifica_para_pcd', 'nivel profissional', 
                  'outro_idioma', 'areas_atuacao', 'principais_atividades',
                  'competencia_tecnicas_e_comportamentais', 'nivel_academico']
padroniza_texto(df_vagas, features_vagas)

# tratamento das colunas de idiomas
language_features = ['nivel_ingles', 'nivel_espanhol']
nivel_idioma(df_applicants, language_features)
nivel_idioma(df_vagas, language_features)

nivel_educacao(df_applicants)
nivel_educacao(df_vagas)


In [16]:
import unicodedata
import re

# tratamento dos regimes de contratação
df_vagas['tipo_contratacao_cleaned'] = df_vagas['tipo_contratacao'].fillna('')
df_vagas['tipo_contratacao_cleaned'] = df_vagas['tipo_contratacao_cleaned'].apply(
    lambda x: [item.strip() for item in x.split(',') if item.strip()]
)

# listando tipos únicos de contratação
tipos_contratacao = set()
for tipo in df_vagas['tipo_contratacao_cleaned']:
    for t in tipo:
        tipos_contratacao.add(t)

# criando colunas binárias
for tipo in tipos_contratacao:
    tipo_formatado = tipo.lower()
    tipo_formatado = unicodedata.normalize('NFKD', tipo_formatado).encode('ascii', 'ignore').decode('utf-8')
    tipo_formatado = re.sub(r'[^a-zA-Z0-9\s]', '', tipo_formatado)
    tipo_coluna = tipo_formatado.replace('/', '_').replace(' ', '_')

    df_vagas[f'contratacao_{tipo_coluna}'] = df_vagas['tipo_contratacao_cleaned'].apply(
        lambda x: 1 if tipo in x else 0
    )

# drop colunas auxiliares
df_vagas = df_vagas.drop(['tipo_contratacao', 'tipo_contratacao_cleaned'], axis=1)


In [17]:
# tratamento de vagas binárias
df_applicants['df_vagas'] = df_applicants['pcd'].apply(lambda x: 1 if 'Sim' else 0)
df_vagas['vaga_sap'] = df_vagas['vaga_sap'].apply(lambda x: 1 if 'Sim' else 0)
df_vagas['vaga_especifica_para_pcd'] = df_vagas['vaga_especifica_para_pcd'].apply(lambda x: 1 if 'Sim' else 0)
df_vagas['local_trabalho'] = df_vagas['local_trabalho'].apply(lambda x: '0' if 'nan' else x)

In [18]:
df_prospects['target_var'] = df_prospects['situacao_candidado'].apply(lambda x: 1 if x in ['Proposta Aceita', 'Aprovado', 'Encaminhar Proposta'] else 0)
df_prospects.drop(columns=['situacao_candidado'])

,id_vaga,titulo,modalidade,nome,id_cand,data_candidatura,ultima_atualizacao,comentario,recrutador,target_var
0,4530,CONSULTOR CONTROL M,,José Vieira,25632,25-03-2021,25-03-2021,"Encaminhado para - PJ R$ 72,00/hora",Ana Lívia Moreira,0
1,4530,CONSULTOR CONTROL M,,Srta. Isabela Cavalcante,25529,22-03-2021,23-03-2021,"encaminhado para - R$ 6.000,00 – CLT Full , n...",Ana Lívia Moreira,0
2,4531,2021-2607395-PeopleSoft Application Engine-Dom...,,Sra. Yasmin Fernandes,25364,17-03-2021,12-04-2021,Data de Inicio: 12/04/2021,Juliana Cassiano,0
3,4531,2021-2607395-PeopleSoft Application Engine-Dom...,,Alexia Barbosa,25360,17-03-2021,17-03-2021,,Juliana Cassiano,0
4,4532,,,NaN,NaN,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...
56697,14219,,,NaN,NaN,NaN,NaN,NaN,NaN,0
56698,14220,Consultor Sênior Especialista SAP LES-TRA - 1433,,Ana Cardoso,16828,26-02-2025,28-02-2025,Recebeu a confirmação de outro processo seleti...,Elisa Nunes,0
56699,14220,Consultor Sênior Especialista SAP LES-TRA - 1433,,Pedro Lucas das Neves,15042,28-02-2025,28-02-2025,,Elisa Nunes,0
56700,14221,Consultor Sênior Oracle EPM FCCS - 1434,,Maria Eduarda Cassiano,49190,26-02-2025,26-02-2025,,Luna Correia,0


In [19]:
# ajuste de nomes de colunas e merge dos dataframe
df_vagas_final = df_vagas.add_suffix('_vaga')
df_applicants_final = df_applicants.add_suffix('_cand')

df_prospects['id_cand'] = df_prospects['id_cand'].astype(str)
df_applicants_final['id_cand'] = df_applicants_final['id_cand'].astype(str)

df_merged = pd.merge(df_prospects, df_applicants_final, on = 'id_cand', how = 'left')
df_merged = pd.merge(df_merged, df_vagas_final, on = 'id_vaga', how = 'left')

In [20]:
df_merged.head()

,id_vaga,titulo,modalidade,nome,id_cand,situacao_candidado,data_candidatura,ultima_atualizacao,comentario,recrutador,target_var,pcd_cand,area_atuacao_cand,conhecimentos_tecnicos_cand,certificacoes_cand,outras_certificacoes_cand,qualificacoes_cand,experiencias_cand,nivel_ingles_cand,nivel_espanhol_cand,outro_idioma_cand,cursos_cand,cargo_atual_cand,data_admissao_cand,data_ultima_promocao_cand,cv_pt_cand,tempo_exp_cand,nivel_ingles_encoded_cand,nivel_espanhol_encoded_cand,nivel_academico_encoded_cand,df_vagas_cand,titulo_vaga_vaga,vaga_sap_vaga,cliente_vaga,solicitante_cliente_vaga,local_trabalho_vaga,vaga_especifica_para_pcd_vaga,nivel profissional_vaga,nivel_ingles_vaga,nivel_espanhol_vaga,outro_idioma_vaga,areas_atuacao_vaga,principais_atividades_vaga,competencia_tecnicas_e_comportamentais_vaga,nivel_ingles_encoded_vaga,nivel_espanhol_encoded_vaga,nivel_academico_encoded_vaga,contratacao_candidato_podera_escolher_vaga,contratacao_hunting_vaga,contratacao_estagiario_vaga,contratacao_pjautonomo_vaga,contratacao_cooperado_vaga,contratacao_clt_cotas_vaga,contratacao_clt_full_vaga
0,4530,CONSULTOR CONTROL M,,José Vieira,25632,Encaminhado ao Requisitante,25-03-2021,25-03-2021,"Encaminhado para - PJ R$ 72,00/hora",Ana Lívia Moreira,0,Não,administrativa comercial financeiracontrolador...,,,,nan,nan,Nenhum,Nenhum,,nan,nan,NaN,NaN,dados pessoais\nestado civil casado\nidade 33 ...,0.0,0.0,0.0,4.0,1.0,reginal service manager portuguese english,1.0,jenkinswalker,ian carvalho,0,1.0,senior,Avançado,Nenhum,,ti projetos,act as regional service deliver manager with ...,sao paulosp,3.0,0.0,8.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,4530,CONSULTOR CONTROL M,,Srta. Isabela Cavalcante,25529,Encaminhado ao Requisitante,22-03-2021,23-03-2021,"encaminhado para - R$ 6.000,00 – CLT Full , n...",Ana Lívia Moreira,0,,,,,,nan,nan,Nenhum,Nenhum,,nan,nan,NaN,NaN,solteiro 47 anos\n\nestrada meringuava no 1763...,0.0,0.0,0.0,0.0,1.0,reginal service manager portuguese english,1.0,jenkinswalker,ian carvalho,0,1.0,senior,Avançado,Nenhum,,ti projetos,act as regional service deliver manager with ...,sao paulosp,3.0,0.0,8.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,4531,2021-2607395-PeopleSoft Application Engine-Dom...,,Sra. Yasmin Fernandes,25364,Contratado pela Decision,17-03-2021,12-04-2021,Data de Inicio: 12/04/2021,Juliana Cassiano,0,,,,,,nan,nan,Nenhum,Nenhum,,nan,nan,NaN,NaN,area de atuacao lider de consultoria gerencia...,0.0,0.0,0.0,0.0,1.0,sap mmwm sr 632022,1.0,morrison ltd,nina rodrigues,0,1.0,pleno,Nenhum,Nenhum,,gestao e alocacao de recursos de ti,conhecimento de processos logisticos compras a...,desenvolvimento de projetos de melhorias e imp...,0.0,0.0,6.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,4531,2021-2607395-PeopleSoft Application Engine-Dom...,,Alexia Barbosa,25360,Encaminhado ao Requisitante,17-03-2021,17-03-2021,,Juliana Cassiano,0,,,,,,nan,nan,Nenhum,Nenhum,,nan,nan,NaN,NaN,informacoes pessoais\n estado civil casado\n n...,0.0,0.0,0.0,0.0,1.0,sap mmwm sr 632022,1.0,morrison ltd,nina rodrigues,0,1.0,pleno,Nenhum,Nenhum,,gestao e alocacao de recursos de ti,conhecimento de processos logisticos compras a...,desenvolvimento de projetos de melhorias e imp...,0.0,0.0,6.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,4532,,,NaN,nan,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,sap co sr 622022,1.0,morrison ltd,nina rodrigues,0,1.0,pleno,Nenhum,Nenhum,,gestao e alocacao de recursos de ti,implementacao de empresa brasil em s4 hana glo...,ingles fluente\n\nrevisao de documentacao bpd ...,0.0,0.0,6.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


# Word2Vec

In [21]:
%pip install gensim

from gensim.models import KeyedVectors
import numpy as np
import pandas as pd
from typing import List

import numpy
import scipy
import gensim
from gensim.models import KeyedVectors

print("NumPy:", numpy.__version__)
print("SciPy:", scipy.__version__)
print("Gensim:", gensim.__version__)



# utilização de modelo word2vec pré-treinado
model = KeyedVectors.load_word2vec_format('word2vec/cbow_s50.txt')

def document_vector(text: str, model: KeyedVectors, num_features: int) -> np.ndarray:
    # Divide o texto em palavras e filtra as que estão no vocabulário do modelo
    if not isinstance(text, str) or not text.strip():
        return np.zeros(num_features) # Retorna um vetor de zeros para texto vazio ou não-string

    words = [word for word in text.split() if word in model.key_to_index]

    if not words:
        return np.zeros(num_features)
    # Calcula a média dos vetores das palavras no documento
    return np.mean([model[word] for word in words], axis=0)


def expand_vector(df: pd.DataFrame, feature_list: List[str], model: KeyedVectors, num_features: int) -> pd.DataFrame:
    df_embeddings = pd.DataFrame()
    # criação de nomes para as colunas
    for feature in feature_list:
        df[f'{feature}_embedding'] = df[f'{feature}'].apply(lambda x: document_vector(x, model, num_features))
        new_embedding_columns = [f'{feature}_{i}' for i in range(num_features)]
        df_embeddings_expanded = pd.DataFrame(
            df[f'{feature}_embedding'].tolist(), # Converte a Series de arrays para uma lista de listas
            columns=new_embedding_columns,        # Atribui os nomes das colunas
            index=df.index                         # Alinha pelo índice
        )
        df.drop(columns = [f'{feature}', f'{feature}_embedding'], inplace=True)
        df_embeddings = pd.concat([df_embeddings, df_embeddings_expanded], axis=1)
    return df_embeddings
    #return pd.concat([df_embeddings, df_embeddings_expanded], axis=1)

Note: you may need to restart the kernel to use updated packages.
NumPy: 1.26.4
SciPy: 1.13.1
Gensim: 4.3.3


In [22]:
text_features_list = ['titulo', 'modalidade', 'nome', 'comentario', 'recrutador',
                      'area_atuacao_cand', 'conhecimentos_tecnicos_cand',
                      'certificacoes_cand', 'outras_certificacoes_cand', 
                      'qualificacoes_cand', 'experiencias_cand', 
                      'cv_pt_cand', 'titulo_vaga_vaga', 'cliente_vaga',
                      'solicitante_cliente_vaga', 'nivel profissional_vaga',
                      'outro_idioma_vaga', 'areas_atuacao_vaga',
                      'principais_atividades_vaga', 'competencia_tecnicas_e_comportamentais_vaga']
df_embeddings = expand_vector(df = df_merged, feature_list = text_features_list, model = model, num_features = 50)
df_final = pd.concat([df_merged, df_embeddings], axis=1)

## Treinamento de Modelo Simples

In [23]:
to_cancel_list = ['situacao_candidado', 'pcd_cand', 'nivel_ingles_cand', 'nivel_espanhol_cand',
                  'outro_idioma_cand', 'cursos_cand', 'cargo_atual_cand', 'data_admissao_cand',
                  'data_ultima_promocao_cand', 'nivel_ingles_vaga', 'nivel_espanhol_vaga',
                  'data_candidatura', 'ultima_atualizacao']
df_final.drop(columns = to_cancel_list, inplace = True)

In [24]:
df_final = df_final.dropna()

In [25]:
# divisao dos dados em treino e teste
X = df_final.drop(columns = ['target_var'])
y = df_final['target_var']

In [26]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [27]:
%pip install matplotlib seaborn


import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# função para rodar o modelo e gerar as métricas de avaliação
def evaluation(model, x_train, y_train, x_test, y_test):
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='micro')
    recall = recall_score(y_test, y_pred, average='micro')
    f1 = f1_score(y_test, y_pred, average='micro')

    print(f'Acurácia: {accuracy}\nPrecisão: {precision}\nRecall: {recall}\nf1-score: {f1}')
    return

Note: you may need to restart the kernel to use updated packages.


In [28]:
from sklearn.linear_model import LogisticRegression

# Criação e Treinamento do modelo
log_regression = LogisticRegression()
evaluation(log_regression, X_train, y_train, X_test, y_test)

/home/diogo/Fase5/.venv/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:470: ConvergenceWarning: lbfgs failed to converge after 100 iteration(s) (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=100).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Acurácia: 0.9953897736797989
Precisão: 0.9953897736797989
Recall: 0.9953897736797989
f1-score: 0.9953897736797989


In [29]:
from sklearn.neighbors import KNeighborsClassifier

# Criação e treinamento do modelo
knn_model = KNeighborsClassifier(n_neighbors=5)
evaluation(knn_model, X_train, y_train, X_test, y_test)

Acurácia: 0.9953897736797989
Precisão: 0.9953897736797989
Recall: 0.9953897736797989
f1-score: 0.9953897736797989


In [30]:
from sklearn.tree import DecisionTreeClassifier

decision_tree = DecisionTreeClassifier()
evaluation(decision_tree, X_train, y_train, X_test, y_test)

Acurácia: 0.9899413243922883
Precisão: 0.9899413243922883
Recall: 0.9899413243922883
f1-score: 0.9899413243922883


In [31]:
from sklearn.svm import SVC

svc_model = SVC()
evaluation(svc_model, X_train, y_train, X_test, y_test)

Acurácia: 0.9953897736797989
Precisão: 0.9953897736797989
Recall: 0.9953897736797989
f1-score: 0.9953897736797989


Salvar o modelo Word2Vec

In [45]:
from pathlib import Path

# Caminho destino para salvar o modelo
model_dir = Path("../src/models")
model_dir.mkdir(parents=True, exist_ok=True)

# Caminho do arquivo de saída
output_path = model_dir / "word2vec_model.kv"

# Salva o modelo no formato recomendado do Gensim
model.save(str(output_path))

print(f"Modelo salvo com sucesso em: {output_path}")


Modelo salvo com sucesso em: ../src/models/word2vec_model.kv


In [46]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('clf', LogisticRegression())
])
pipeline.fit(X_train, y_train)  # aqui você treina seu modelo


,steps,"[('scaler', ...), ('clf', ...)]"
,transform_input,None
,memory,None
,verbose,False
,copy,True
,with_mean,True
,with_std,True
,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0


In [ ]:
import joblib
from pathlib import Path

# supondo que `pipeline` seja o objeto retornado pelo seu treinamento
artifacts_dir = Path.cwd() / 'models' / 'artifacts'
artifacts_dir.mkdir(parents=True, exist_ok=True)

joblib.dump(pipeline, artifacts_dir / 'pipeline.joblib')
print(f"Pipeline salvo em: {artifacts_dir / 'pipeline.joblib'}")


Pipeline salvo em: /home/diogo/Fase5/TechChallenge_Fase5/src/src/models/artifacts/pipeline.joblib
